In [3]:
import os
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16


In [4]:
dataset_path = 'C:/Users/ritik/Downloads/Compressed/KinFaceW-I/KinFaceW-I'
model_path = 'path/to/model'

In [5]:
batch_size = 32
epochs = 10
learning_rate = 0.001

In [6]:
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
train_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    subset='training'
)
val_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    subset='validation'
)

Found 853 images belonging to 2 classes.
Found 213 images belonging to 2 classes.


In [7]:
base_model = VGG16(include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
    layer.trainable = False

In [8]:
print(base_model.summary())

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
model = Sequential([
    base_model,
    Flatten(),
    Dense(1024, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 1024)              25691136  
                                                                 
 batch_normalization (BatchN  (None, 1024)             4096      
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 1024)              0         
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                        

In [18]:
model.compile(optimizer=Adam(lr=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(
    train_generator,
    epochs=epochs,
    validation_data=val_generator,
    verbose=1
)


Epoch 1/10
27/27 [==============================] - 70s 3s/step - loss: 1.0998 - accuracy: 0.5000 - val_loss: 1.3251 - val_accuracy: 0.5000
Epoch 2/10
27/27 [==============================] - 71s 3s/step - loss: 0.9352 - accuracy: 0.5000 - val_loss: 1.1430 - val_accuracy: 0.5000
Epoch 3/10
27/27 [==============================] - 72s 3s/step - loss: 0.8822 - accuracy: 0.5000 - val_loss: 1.0090 - val_accuracy: 0.5000
Epoch 4/10
27/27 [==============================] - 72s 3s/step - loss: 0.8500 - accuracy: 0.5000 - val_loss: 0.7397 - val_accuracy: 0.5000
Epoch 5/10
27/27 [==============================] - 72s 3s/step - loss: 0.8218 - accuracy: 0.5000 - val_loss: 0.7093 - val_accuracy: 0.5000
Epoch 6/10
27/27 [==============================] - 73s 3s/step - loss: 0.8077 - accuracy: 0.5000 - val_loss: 0.8512 - val_accuracy: 0.5000
Epoch 7/10
27/27 [==============================] - 75s 3s/step - loss: 0.7857 - accuracy: 0.5000 - val_loss: 0.7325 - val_accuracy: 0.5000
Epoch 8/10
27/27 [==

In [19]:
test_generator = datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=batch_size,
    shuffle=False
)
y_pred = model.predict(test_generator)
y_pred = np.round(y_pred).flatten()
y_true = test_generator.classes
accuracy = np.mean(y_pred == y_true)
print(f'Test accuracy: {accuracy:.2f}')


Found 1066 images belonging to 2 classes.
34/34 [==============================] - 70s 2s/step
Test accuracy: 0.65
